In [11]:
import numpy as np
import pennylane as qml

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split


In [12]:
# Load dataset
data = fetch_openml(name="ionosphere", version=1, as_frame=False)
X, y = data.data, data.target

# Encode labels: g -> 1, b -> -1
y = np.array([1 if label == "g" else -1 for label in y])

# Normalize features
X = StandardScaler().fit_transform(X)

# PCA: 34 → 4 features
pca = PCA(n_components=4)
X = pca.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [13]:
n_qubits = 4
n_layers = 2

dev = qml.device("default.qubit", wires=n_qubits)


In [14]:
def encode_features(x):
    for i in range(n_qubits):
        qml.RY(np.pi * x[i], wires=i)

def variational_circuit(weights):
    for layer in range(n_layers):
        for q in range(n_qubits):
            qml.RY(weights[layer, q, 0], wires=q)
            qml.RZ(weights[layer, q, 1], wires=q)

        # Entanglement
        for q in range(n_qubits - 1):
            qml.CNOT(wires=[q, q + 1])



In [35]:
@qml.qnode(dev)
def vqc(x, weights):
    encode_features(x)
    variational_circuit(weights)

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


In [36]:
def predict(x, weights):
    return np.mean(vqc(x, weights))

def loss(weights, X, y):
    preds = np.array([vqc(x, weights) for x in X])
    return np.mean(np.log(1 + np.exp(-y * preds)))

def accuracy(X, y, weights):
    preds = np.sign([vqc(x, weights) for x in X])
    return np.mean(preds == y)



In [37]:
np.random.seed(42)

# Weights shape: (layers, qubits, params)
weights = np.random.normal(0, 0.1, size=(n_layers, n_qubits, 2))

optimizer = qml.AdamOptimizer(stepsize=0.05)
epochs = 40

for epoch in range(epochs):
    weights = optimizer.step(lambda w: loss(w, X_train, y_train), weights)

    if epoch % 5 == 0:
        train_loss = loss(weights, X_train, y_train)
        print(f"Epoch {epoch:02d} | Loss: {train_loss:.4f}")


ValueError: operands could not be broadcast together with shapes (280,) (280,4) 

In [34]:
train_acc = accuracy(X_train, y_train, weights)
test_acc = accuracy(X_test, y_test, weights)

print("\nFinal Results")
print("Train Accuracy:", train_acc)
print("Test Accuracy:", test_acc)



Final Results
Train Accuracy: 0.48928571428571427
Test Accuracy: 0.4507042253521127
